In [1]:
from pulp import *

# Création du problème
prob = LpProblem("Optimisation", LpMaximize)

# Variables de décision
x = LpVariable.dicts("x", range(1, 5), lowBound=0)
y = LpVariable.dicts("y", range(1, 5), cat=LpBinary)
z = LpVariable("z", cat=LpBinary)

# Coefficients de la fonction objectif
c = [70, 60, 90, 80, -50000, -40000, -70000, -60000]

# Fonction objectif
prob += lpDot(c, [x[i] for i in range(1, 5)] + [y[i] for i in range(1, 5)]) - z * 6000, "Profit_total"

# Contraintes
prob += lpSum([4 * y[1]]) <= 2, "Contrainte1"
for i in range(1, 5):
    prob += x[i] <= 1000000 * y[i], f"Contrainte2_{i}"
prob += y[3] + y[4] <= y[1] + y[2], "Contrainte3"
prob += lpDot([5, 3, 6, 4], [x[i] for i in range(1, 5)]) <= 6000 + z * 1000000, "Contrainte4"
prob += lpDot([4, 6, 3, 5], [x[i] for i in range(1, 5)]) <= 6000 + (1 - z) * 1000000, "Contrainte5"

# Résolution du problème
prob.solve()

# Affichage du statut de la solution
print("Statut:", LpStatus[prob.status])

# Affichage des valeurs des variables
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Affichage de la valeur de la fonction objectif
print("Profit total =", value(prob.objective))


Statut: Optimal
x_1 = 0.0
x_2 = 2000.0
x_3 = 0.0
x_4 = 0.0
y_1 = 0.0
y_2 = 1.0
y_3 = 0.0
y_4 = 0.0
z = 0.0
Profit total = 80000.0


In [3]:
import numpy as np
from scipy.optimize import linprog

# Coefficients de la fonction objectif
c = [-70, -60, -90, -80, 50000, 40000, 70000, 60000]

# Matrice des contraintes
A = np.array([
    [0, 0, 0, 0, -4, 0, 0, 0],
    [-1, 0, 0, 0, 0, 0, 0, 0],
    [0, -1, 0, 0, 0, 0, 0, 0],
    [0, 0, -1, 0, 0, 0, 0, 0],
    [0, 0, 0, -1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -1, 0],
    [0, 0, 0, 0, 0, 0, 0, -1],
    [0, 0, 0, 0, 0, 1, 1, -1],
    [-5, -3, -6, -4, 0, 0, 0, 0],
    [-4, -6, -3, -5, 0, 0, 0, 0]
])

# Vecteur des limites des contraintes
b = np.array([-2, 0, 0, 0, 0, 0, 0, 0, -6000, -6000])

# Bornes des variables
x_bounds = [(0, None)] * 4
y_bounds = [(0, 1)] * 4
z_bounds = [(0, 1)]

# Résolution du problème
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds + y_bounds + z_bounds, method='highs')

# Affichage du statut de la solution
print("Statut:", result.message)

# Affichage des valeurs des variables
print("Valeurs des variables:")
for i, var in enumerate(result.x):
    print(f"x{i+1} =", var)

# Affichage de la valeur de la fonction objectif
print("Profit total =", -result.fun)


ValueError: Invalid input for linprog: unable to interpret bounds with this dimension tuple: (9, 2).